#### Sequence classification

Is a predictive modeling problem where you have some sequence of inputs over space or time and the task is to predict a category for the sequence.

What makes this problem difficult is that the sequences can vary in length, be comprised of a very large vocabulary of input symbols and may require the model to learn the long-term context or dependencies between symbols in the input sequence.

##### Problem Description

IMDB movie review sentiment classification problem.

    The Large Movie Review Dataset (often referred to as the IMDB dataset) contains 25,000 highly-polar movie reviews (good or bad) for training and the same amount again for testing. 
    The problem is to determine whether a given movie review has a positive or negative sentiment.

Keras comes with in-built IMDB dataset. The __imdb.load_data()__ function allows you to load the dataset in a format that is ready for use in neural network and deep learning models.

## Simple LSTM for Sequence Classification

#### Importing the classes and functions required 

### Load the dependencies

In [1]:
# LSTM for sequence classification in the IMDB dataset
import numpy

from keras.datasets import imdb #A utility to load a dataset

from keras.models import Sequential

from keras.layers import Dense, LSTM, Dropout, Embedding
from keras.layers import Conv1D, MaxPooling1D

from keras.preprocessing import sequence #To convert a variable length sentence into a prespecified length

# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest

Using TensorFlow backend.
/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


##### Load the IMDB dataset 

In [2]:
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

17465344/17464789 [==============================] - 77s 4us/step


##### Let's inspect the first one sentences of train and their classes

In [3]:
print(X_train[:1])
print(y_train[:1])

[ list([1, 14, 20, 100, 28, 77, 6, 542, 503, 20, 48, 342, 470, 7, 4, 4, 20, 286, 38, 76, 4085, 23, 4, 383, 139, 13, 384, 240, 6, 383, 2, 5, 146, 252, 15, 225, 6, 176, 53, 15, 271, 23, 19, 383, 2, 1005, 7, 260, 383, 23, 6, 1813, 2857, 488, 2, 2, 122, 6, 52, 292, 1069, 51, 32, 29, 69, 8, 81, 63, 286, 76, 33, 31, 213, 42, 160, 31, 62, 28, 8, 462, 33, 90, 88, 27, 109, 16, 38, 2, 2, 2, 16, 2659, 11, 41, 217, 17, 4, 1947, 383, 2, 59, 2856, 7, 224, 53, 151, 5, 146, 24, 2, 41, 260, 383, 4, 415, 15, 3405, 46, 4, 91, 8, 72, 11, 14, 20, 16, 2, 2, 11, 41, 1078, 217, 17, 4, 1715, 5, 1947, 322, 225, 142, 44, 307, 1004, 5, 46, 15, 2303, 2, 8, 72, 59, 256, 15, 217, 5, 17, 25, 296, 4, 20, 25, 380, 8, 235, 78, 18, 41, 10, 10, 2, 7, 6, 383, 2, 137, 24, 735, 819, 42, 6, 682, 356, 8, 2, 1553, 9, 179, 2, 5, 127, 6, 1257, 292, 11, 800, 25, 89, 2066, 965, 2624, 70, 193, 120, 5, 2457, 4, 55, 183, 11, 113, 25, 104, 545, 7])]
[0]


Truncate and pad the input sequences so that they are all the same length for modeling. 

The model will learn the zero values carry no information so indeed the sequences are not the same length in terms of content, but same length vectors is required to perform the computation in Keras.

In [4]:
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

##### Inspect the same sentences after padding

In [5]:
print(X_train[0:1], y_train[0:1])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   

The first layer is the Embedded layer that uses 32 length vectors to represent each word. 

The next layer is the LSTM layer with 100 memory units (smart neurons). 

Finally, because this is a classification problem we use a Dense output layer with a single neuron and a sigmoid activation function to make 0 or 1 predictions for the two classes (good and bad) in the problem.

In [6]:
# create the model lstm
embedding_vector_length = 32

model = Sequential()

model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

Because it is a binary classification problem, log loss is used as the loss function (binary_crossentropy in Keras). 

The efficient ADAM optimization algorithm is used. 

In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


The model is fit for only 2 epochs because it quickly overfits the problem. 

A large batch size of 64 reviews is used to space out weight updates.

In [8]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 380s 15ms/step - loss: 0.5240 - acc: 0.7296 - val_loss: 0.3914 - val_acc: 0.8291
Epoch 2/3
25000/25000 [==============================] - 351s 14ms/step - loss: 0.3063 - acc: 0.8752 - val_loss: 0.2945 - val_acc: 0.8786
Epoch 3/3
25000/25000 [==============================] - 353s 14ms/step - loss: 0.2639 - acc: 0.8932 - val_loss: 0.3056 - val_acc: 0.8740


we estimate the performance of the model on unseen reviews


In [9]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)

print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 87.40%


###### LSTM For Sequence Classification With Dropout

LSTM generally have the problem of overfitting.

Dropout can be applied between layers using the Dropout Keras layer. 

    We can do this easily by adding new Dropout layers between the Embedding and LSTM layers and the LSTM and Dense output layers. 

In [10]:
# create the model
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 32)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
model.fit(X_train, y_train, epochs=3, batch_size=64)

Epoch 1/3
25000/25000 [==============================] - 320s 13ms/step - loss: 0.4617 - acc: 0.7700
Epoch 2/3
25000/25000 [==============================] - 286s 11ms/step - loss: 0.3674 - acc: 0.8448
Epoch 3/3
25000/25000 [==============================] - 263s 11ms/step - loss: 0.2688 - acc: 0.8934


In [13]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 87.30%


Observation: 
    
    Dropout having the desired impact on training with a slightly slower trend in convergence. 

Ref:

    https://machinelearningmastery.com

    https://keras.io